In [3]:
from functools import cache


def mix(a, b):
    return a ^ b


def prune(a):
    return a % 16777216


@cache
def calc_secret(number, iteration):
    if iteration == 0:
        return number

    number = mix(number, number * 64)
    number = prune(number)
    number = mix(number, int(number / 32))
    number = prune(number)
    number = mix(number, number * 2048)
    number = prune(number)

    return calc_secret(number, iteration - 1)


@cache
def evolve_secret(number):
    number = mix(number, number * 64)
    number = prune(number)
    number = mix(number, int(number / 32))
    number = prune(number)
    number = mix(number, number * 2048)
    number = prune(number)
    return number


# calc_secret(123, 4)
example = """1: 8685429
10: 4700978
100: 15273692
2024: 8667524"""
example = [list(map(int, x.split(":"))) for x in example.split("\n")]
for i, o in example:
    m = calc_secret(i, 2000)
    print(f"{i} : expected {o} got {m} diff {m - o}")

1 : expected 8685429 got 8685429 diff 0
10 : expected 4700978 got 4700978 diff 0
100 : expected 15273692 got 15273692 diff 0
2024 : expected 8667524 got 8667524 diff 0


In [4]:
input = [int(x.strip()) for x in open("input", "r").readlines()]
sum([calc_secret(x, 2000) for x in input])

13234715490

In [27]:
import pandas as pd
import numpy as np
from tqdm import tqdm

buyers = [int(x.strip()) for x in open("input", "r").readlines()]
res = []
for b in tqdm(buyers):
    loop_res = [(b, b)]
    for _ in range(2000):
        num = evolve_secret(loop_res[-1][1])
        loop_res.append((b, num))
    res.append(loop_res)

100%|██████████| 1557/1557 [00:01<00:00, 1202.95it/s]


In [69]:
df = (
    pd.DataFrame(res)
    .melt(var_name="iter", value_name="res")
    .assign(
        buyer=lambda x: x.res.str[0].astype(int),
        secret=lambda x: x.res.str[1].astype(int),
    )
    .drop(columns="res")
).sort_values(["buyer", "iter"])

In [70]:
df.assign(
    price=lambda x: x.secret.astype(str).str[-1].astype(int),
    change=lambda x: (x.price - x.price.shift()).fillna(0).astype(int),
).assign(
    change_checksum=lambda x: (x.change.shift(3) + 9) * 19**3
    + (x.change.shift(2) + 9) * 19**2
    + (x.change.shift(1) + 9) * 19
    + (x.change.shift(0) + 9)
).assign(
    first_iter=lambda x: x.groupby(["buyer", "change_checksum"]).iter.transform("min")
).query(
    "iter == first_iter"
).groupby(
    "change_checksum", as_index=False
).agg(
    total_price=("price", "sum")
).sort_values(
    "total_price", ascending=False
)

,change_checksum,total_price
23644,71659.0,1490
17057,58322.0,1485
20224,64819.0,1463
11264,44946.0,1449
23965,72076.0,1425
...,...,...
2381,16785.0,0
31400,90715.0,0
29419,84972.0,0
29434,84990.0,0


In [36]:
df.assign(price=lambda x: x.secret.astype(str).str[-1].astype(int)).assign(
    change=lambda x: (x.price - x.price.shift()).fillna(0).astype(int)
).query("iter > 0").assign(
    prev4=lambda x: x.change.shift([3, 2, 1, 0]).apply(
        lambda row: "|".join(row.fillna(-999).astype(int).astype(str)), axis=1
    )
).assign(
    first_iter=lambda x: x.groupby(["buyer", "prev4"]).iter.transform("min")
).query(
    "iter == first_iter"
).groupby(
    "prev4", as_index=False
).agg(
    total_price=("price", "sum")
).sort_values(
    "total_price", ascending=False
).head(
    1
).total_price

KeyboardInterrupt: 